## 원본 json 데이터를 csv로 저장
- 필요한 컬럼만 불러오기

In [5]:
import pandas as pd
import gzip
import json 

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
  

df = getDF('../original_data/amazon_2018/Grocery_and_Gourmet_Food_5.json.gz')

In [3]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,NaN,NaN,NaN
1,5.0,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,NaN,NaN,NaN
2,5.0,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,NaN,NaN,NaN
3,5.0,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,NaN,NaN,NaN
4,5.0,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,NaN,NaN,NaN


In [6]:
len(df)

1143860

In [ ]:
df_p=df[['reviewerID','asin','reviewerName','reviewText','overall','summary','reviewTime']]
df_p.to_csv('preprocessed_data(18)/reviews.csv', index=False)


In [88]:
df_m = getDF('original_data/amazon_2018/meta_Grocery_and_Gourmet_Food.json.gz')
df_m.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Grocery & Gourmet Food, Dairy, Cheese & Eggs,...",,"[BEEMSTER GOUDA CHEESE AGED 18/24 MONTHS, Stat...",,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,[],,Ariola Imports,[],"165,181 in Grocery & Gourmet Food (","[B0000D9MYM, B0000D9MYL, B00ADHIGBA, B00H9OX59...",Grocery,,,$41.91,0681727810,[],[],NaN
1,"[Grocery & Gourmet Food, Cooking & Baking, Sug...",,"[Shipped from UK, please allow 10 to 21 busine...",,Trim Healthy Mama Xylitol,"[B01898YHXK, B01BCM6LAC, B00Q4OL47O, B00Q4OL5Q...",,,[],"315,867 in Grocery & Gourmet Food (",[],Grocery,,,,0853347867,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Grocery & Gourmet Food, Cooking & Baking, Fro...",,[Jazz up your cakes with a sparkling monogram ...,,Letter C - Swarovski Crystal Monogram Wedding ...,[],,Unik Occasions,[],"[>#669,941 in Kitchen & Dining (See Top 100 in...",[B07DXN65TF],Amazon Home,,"September 21, 2010",$29.95,1888861118,[],[],NaN
3,"[Grocery & Gourmet Food, Cooking & Baking, Fro...",,"[Large Letter - Height 4.75""]",,Letter H - Swarovski Crystal Monogram Wedding ...,[],,Other,"[Large Letter - Height 4.75""]","[>#832,581 in Kitchen & Dining (See Top 100 in...",[],Amazon Home,,"September 11, 2011",$11.45,1888861517,[],[],NaN
4,"[Grocery & Gourmet Food, Cooking & Baking, Fro...",,"[4.75""]",,Letter S - Swarovski Crystal Monogram Wedding ...,[],,Unik Occasions,"[4.75"" height]","[>#590,999 in Kitchen & Dining (See Top 100 in...",[],Amazon Home,,"September 11, 2011",$15.00,1888861614,[],[],NaN


In [ ]:
# df_m_p=df_m[['asin','title','categories','price','brand','related']]
df_m_p = df_m[['asin', 'title', 'main_cat', 'price', 'brand','also_buy','also_view','category']]
df_m_p.to_csv('original_data/amazon_2018/meta.csv', index=False)


## 정규화 조건에 맞춰 테이블 분리하기

In [92]:
# 2018 버전 전처리
import pandas as pd

df_m_p=pd.read_csv('original_data/amazon_2018/meta.csv')
df_p=pd.read_csv('original_data/amazon_2018/reviews.csv')

# User 테이블: reviewerID, reviewerName (중복 제거)
user_table = df_p[['reviewerID', 'reviewerName']].drop_duplicates()

# Product 테이블: asin, title, main_cat, price, brand (중복 제거)
product_table = df_m_p[['asin', 'title', 'main_cat', 'price', 'brand','category']].drop_duplicates()

# Review 테이블: 리뷰 고유 ID 추가
df_p['reviewID'] = range(1, len(df_p) + 1)
review_table = df_p[['reviewID', 'reviewerID', 'asin', 'reviewText', 'summary', 'overall', 'reviewTime']]

# AlsoBuy 테이블: asin - also_buy_asin 관계 분리
also_buy_table = df_m_p[['asin', 'also_buy']].dropna()
also_buy_table = also_buy_table.explode('also_buy')
also_buy_table = also_buy_table.rename(columns={'also_buy': 'also_buy_asin'})

# AlsoView 테이블: asin - also_view_asin 관계 분리
also_view_table = df_m_p[['asin', 'also_view']].dropna()
also_view_table = also_view_table.explode('also_view')
also_view_table = also_view_table.rename(columns={'also_view': 'also_view_asin'})


### 전처리 코드
- 중복 행 제거

In [96]:
len(user_table[user_table['reviewerName'].isnull()])
user_table=user_table.dropna(subset=['reviewerName'])  # 리뷰어 이름이 없는 행(24) 제거
user_table

,reviewerID,reviewerName
0,A1QVBUH9E1V6I8,Jamshed Mathur
1,A3GEOILWLK86XM,itsjustme
2,A32RD6L701BIGP,Krystal Clifton
3,A2UY1O1FBGKIE6,U. Kane
4,A3QHVBQYDV7Z6U,The Nana
...,...,...
1143271,A38YATKTY2TNPJ,Noman
1143275,A1KI9J8XEIN8V1,Shopper Extrodinaire
1143603,A1KD3XXEGDTMXC,Stewardship
1143624,A3N3K1C317YP52,Ami Francesconi


In [ ]:
user_table=user_table.groupby('reviewerID', as_index=False).last()  # reviewerID에 reviewerName 중복 경우 존재 -> reviewerID별로 마지막 리뷰어 이름을 가져옴
user_table  #127473

,reviewerID,reviewerName
0,A0096681Y127OL1H8W3U,Robert L.
1,A0205727NVAUDXS07YYO,1SG
2,A0240447UBWOLI8KGOGT,Debmun1215
3,A0261431Y0V4MHWY4B7W,BoJess
4,A0281074R0QWE993EX2F,Savy94
...,...,...
127468,AZZT45L993PLR,Amazon Customer
127469,AZZV9PDNMCOZW,Anne B. Ward
127470,AZZXT5AZPVZBL,A. Blumenthal
127471,AZZYW4YOE1B6E,tmybrgr


In [107]:
review_table=review_table[review_table['reviewText'].notna() & (review_table['reviewText'].str.strip() != '')] # revieText가 없는 경우 415 제거 (1143445)
review_table

,reviewID,reviewerID,asin,reviewText,summary,overall,reviewTime
0,1,A1QVBUH9E1V6I8,4639725183,No adverse comment.,Five Stars,5.0,"11 19, 2014"
1,2,A3GEOILWLK86XM,4639725183,Gift for college student.,Great product.,5.0,"10 13, 2016"
2,3,A32RD6L701BIGP,4639725183,"If you like strong tea, this is for you. It mi...",Strong,5.0,"11 21, 2015"
3,4,A2UY1O1FBGKIE6,4639725183,Love the tea. The flavor is way better than th...,Great tea,5.0,"08 12, 2015"
4,5,A3QHVBQYDV7Z6U,4639725183,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,5.0,"05 28, 2015"
...,...,...,...,...,...,...,...
1143855,1143856,A223YRQH2Z5T1D,B01HJF6FRA,"As a new vegan, it is sometimes difficult to r...","As a new vegan, it is sometimes difficult to ...",5.0,"09 8, 2017"
1143856,1143857,A38GDA4TB9EILT,B01HJF6FRA,The best thing ever is ordering a product you ...,The best thing ever is ordering a product you ...,5.0,"08 4, 2017"
1143857,1143858,A2025PN7HDC5BO,B01HJF6FRA,I used to love ranch before I became vegan. It...,Just what the vegan ordered!,5.0,"07 4, 2017"
1143858,1143859,A1NY7XWC7EPQOA,B01HJF6FRA,I cannot have dairy nor gluten. This is as cl...,This is as close to Ranch as I will ever be ab...,5.0,"06 7, 2017"


In [106]:
product_table[product_table['brand'].isnull()]  # title이 없는 경우 3, main_cat이 없는 경우 1363, price가 없는 경우 151463, brand가 없는 경우 11264

,asin,title,main_cat,price,brand,category
1,0853347867,Trim Healthy Mama Xylitol,Grocery,NaN,NaN,"['Grocery & Gourmet Food', 'Cooking & Baking',..."
36,9178894018,IKEA - BRYGGKAFFE MELLANROST Decaffeinated Cof...,Grocery,.a-box-inner{background-color:#fff}#alohaBuyBo...,NaN,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."
81,B00008RCMI,"Dentyne Ice Sugarless Gum, Peppermint - 12x4 =...",Grocery,$45.00,NaN,"['Grocery & Gourmet Food', 'Candy & Chocolate'..."
83,B000096O8Y,Necco Assorted Wafers (2 boxes with 36 rolls e...,Grocery,NaN,NaN,"['Grocery & Gourmet Food', 'Candy & Chocolate'..."
108,B0000CD07U,"Ketchup, Fruit Sweetened, 14 oz.",Grocery,NaN,NaN,"['Grocery & Gourmet Food', 'Condiments & Salad..."
...,...,...,...,...,...,...
286781,B01HFJUY4U,DECAF ESPRESSO Whole Bean Coffee | 1 LB Bag,Grocery,NaN,NaN,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."
286782,B01HFJUYNQ,ESPRESSO DECAFFEINATO Decaf Whole Bean Coffee ...,Grocery,NaN,NaN,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."
286783,B01HFJZEC2,Stone Street Coffee Colombian Supremo Fresh Ro...,Grocery,NaN,NaN,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."
286803,B01HFMVHK2,French`s Buffalo Ketchup 20 oz Bottle (Pack of 2),Grocery,$20.12,NaN,"['Grocery & Gourmet Food', 'Condiments & Salad..."


In [ ]:
product_table = product_table.dropna(subset=['title'])  # title이 없는 행(3) 제거
import re

def is_invalid_price(text):
    if not isinstance(text, str):
        return False
    pattern = r'\.a-box-inner|\{|\}|background-color|#\w+'
    return re.search(pattern, text) is not None

# 조건 만족하는 행 제거
product_table=product_table[~product_table['price'].apply(is_invalid_price)] # price가 잘못 긁힌 경우(1657) 제거 (예:.a-box-inner{background-color:#fff}#alohaButton) 

product_table=product_table[product_table['main_cat']=='Grocery']  # main_cat이 Grocery가 아닌 경우(24795) 제거
product_table

,asin,title,main_cat,price,brand,category
0,0681727810,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,Grocery,$41.91,Ariola Imports,"['Grocery & Gourmet Food', 'Dairy, Cheese & Eg..."
1,0853347867,Trim Healthy Mama Xylitol,Grocery,NaN,NaN,"['Grocery & Gourmet Food', 'Cooking & Baking',..."
2,1888861118,Letter C - Swarovski Crystal Monogram Wedding ...,Amazon Home,$29.95,Unik Occasions,"['Grocery & Gourmet Food', 'Cooking & Baking',..."
3,1888861517,Letter H - Swarovski Crystal Monogram Wedding ...,Amazon Home,$11.45,Other,"['Grocery & Gourmet Food', 'Cooking & Baking',..."
4,1888861614,Letter S - Swarovski Crystal Monogram Wedding ...,Amazon Home,$15.00,Unik Occasions,"['Grocery & Gourmet Food', 'Cooking & Baking',..."
...,...,...,...,...,...,...
287046,B01HJFBN9A,"Caffarel Fruit Jellies, Organic Assorted, 200 ...",Grocery,NaN,Caffarel,"['Grocery & Gourmet Food', 'Jams, Jellies & Sw..."
287047,B01HJFC704,"HAMPTON CREEK Spo Just Truffle Mayo, 12 Ounce",Grocery,$10.89,Hampton Creek,"['Grocery & Gourmet Food', 'Condiments & Salad..."
287048,B01HJF6FRA,"HAMPTON CREEK Just Ranch Dressing, 12 Fluid Ounce",Grocery,$10.95,Hampton Creek,"['Grocery & Gourmet Food', 'Condiments & Salad..."
287049,B01HJHSVG6,Gourmet Pure Vanilla Bean Pastes - Organically...,Grocery,$14.97,Heilala Vanilla,"['Grocery & Gourmet Food', 'Herbs, Spices & Se..."


In [133]:
import ast

def get_second_non_grocery_category(x):
    try:
        # 문자열 → 리스트 변환
        lst = ast.literal_eval(x) if isinstance(x, str) else []
        # 'Grocery&Gourmet Food' 제거
        filtered = [item for item in lst if item != 'Grocery&Gourmet Food']
        # 두 번째 요소 추출 (인덱스 1)
        return filtered[1] if len(filtered) > 1 else (filtered[0] if len(filtered) == 1 else None)
    except Exception:
        return None

# 적용
product_table['category']=product_table['category'].apply(get_second_non_grocery_category)
product_table

/var/folders/wj/tbypyq6j5pn9209l5xv_vrz40000gn/T/ipykernel_31847/3854191845.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_table['category']=product_table['category'].apply(get_second_non_grocery_category)


,asin,title,main_cat,price,brand,category
0,0681727810,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,Grocery,$41.91,Ariola Imports,"Dairy, Cheese & Eggs"
1,0853347867,Trim Healthy Mama Xylitol,Grocery,NaN,NaN,Cooking & Baking
10,4639725043,Lipton Yellow Label Tea (loose tea) - 450g,Grocery,$12.46,Lipton,Beverages
11,4639725183,Lipton Yellow Label Finest Blend Tea Bags 100 ...,Grocery,$12.98,Lipton,Beverages
12,4858582000,Wagh Bakri Masala Chai 100's (Pack of 3),Grocery,$25.93,Wagh Bakri,Beverages
...,...,...,...,...,...,...
287046,B01HJFBN9A,"Caffarel Fruit Jellies, Organic Assorted, 200 ...",Grocery,NaN,Caffarel,"Jams, Jellies & Sweet Spreads"
287047,B01HJFC704,"HAMPTON CREEK Spo Just Truffle Mayo, 12 Ounce",Grocery,$10.89,Hampton Creek,Condiments & Salad Dressings
287048,B01HJF6FRA,"HAMPTON CREEK Just Ranch Dressing, 12 Fluid Ounce",Grocery,$10.95,Hampton Creek,Condiments & Salad Dressings
287049,B01HJHSVG6,Gourmet Pure Vanilla Bean Pastes - Organically...,Grocery,$14.97,Heilala Vanilla,"Herbs, Spices & Seasonings"


In [ ]:
import ast

# also_buy_asin 컬럼 처리
also_buy_table['also_buy_asin'] = also_buy_table['also_buy_asin'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else []) # 287051

# 빈 리스트 제거
also_buy_table=also_buy_table[also_buy_table['also_buy_asin'].apply(lambda x: isinstance(x, list) and len(x) > 0)] 

# also_view_asin 컬럼 처리
also_view_table['also_view_asin'] = also_view_table['also_view_asin'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])  # 166113

# 빈 리스트 제거
also_view_table=also_view_table[also_view_table['also_view_asin'].apply(lambda x: isinstance(x, list) and len(x) > 0)]  


In [ ]:
also_buy_table # 83294
also_view_table # 120938
user_table # 130529  -> 127473
product_table  # 256901
review_table # 1143445

,asin,title,main_cat,price,brand
0,0681727810,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,Grocery,$41.91,Ariola Imports
1,0853347867,Trim Healthy Mama Xylitol,Grocery,NaN,NaN
2,1888861118,Letter C - Swarovski Crystal Monogram Wedding ...,Amazon Home,$29.95,Unik Occasions
3,1888861517,Letter H - Swarovski Crystal Monogram Wedding ...,Amazon Home,$11.45,Other
4,1888861614,Letter S - Swarovski Crystal Monogram Wedding ...,Amazon Home,$15.00,Unik Occasions
...,...,...,...,...,...
283349,B01HJFBN9A,"Caffarel Fruit Jellies, Organic Assorted, 200 ...",Grocery,NaN,Caffarel
283350,B01HJFC704,"HAMPTON CREEK Spo Just Truffle Mayo, 12 Ounce",Grocery,$10.89,Hampton Creek
283351,B01HJF6FRA,"HAMPTON CREEK Just Ranch Dressing, 12 Fluid Ounce",Grocery,$10.95,Hampton Creek
283352,B01HJHSVG6,Gourmet Pure Vanilla Bean Pastes - Organically...,Grocery,$14.97,Heilala Vanilla


In [52]:
import pandas as pd

also_buy_table=pd.read_csv('/Users/yejin/Desktop/also_buy.csv')

normalized_df = also_buy_table.explode('asin').rename(columns={'also_buy_list': 'also_bought_asin'})

# 결과 확인
print(normalized_df)

             asin                                      also_buy_asin
0      0853347867  ['B01898YHXK', 'B01BCM6LAC', 'B00Q4OL47O', 'B0...
1      4639725043  ['B00886E4K0', 'B00CREXSHY', 'B001QTRGAQ', 'B0...
2      4639725183  ['B000JSQDK4', 'B007ZI1SKG', '4639725043', 'B0...
3      5236363640  ['B002FBD900', 'B00015YTS8', 'B00B3ARYDC', 'B0...
4      5463213682  ['B00F3XJX6G', 'B003CY45VG', 'B003CGJAIM', 'B0...
...           ...                                                ...
83289  B01HJEO4QK         ['B01HJEHE9E', 'B01HJF6FRA', 'B01N6LF9LZ']
83290  B01HJFBN9A  ['B01HULC3TC', 'B07C8L4NWL', 'B014UTG6D6', 'B0...
83291  B01HJFC704  ['B00WNAEX2Q', 'B071LD7PMJ', 'B01HJF6FRA', 'B0...
83292  B01HJF6FRA  ['B071LD7PMJ', 'B00KOHNX6C', 'B00OVN2F64', 'B0...
83293  B01HJHSVG6  ['B00DUF219A', 'B01HJHX9ZO', 'B07K4K26PQ', 'B0...

[83294 rows x 2 columns]


In [135]:
user_table=user_table.reset_index(drop=True)
product_table=product_table.reset_index(drop=True)
review_table=review_table.reset_index(drop=True)
also_buy_table=also_buy_table.reset_index(drop=True)
also_view_table=also_view_table.reset_index(drop=True)

In [137]:
user_table.to_csv("preprocessed_data(18)/users.csv", index=False)
product_table.to_csv("preprocessed_data(18)/products.csv",index=False)
review_table.to_csv("preprocessed_data(18)/reviews.csv", index=False)
also_buy_table.to_csv("preprocessed_data(18)/also_buy.csv", index=False)
also_view_table.to_csv("preprocessed_data(18)/also_view.csv", index=False)

In [48]:
import csv
import json

# CSV 파일 경로와 JSON 파일 경로를 지정
csv_file = 'preprocessed_data(18)/user.csv'
json_file = 'preprocessed_data(18)/users.json'

# CSV 파일을 읽고 JSON 파일로 변환
with open(csv_file, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

with open(json_file, 'w', encoding='utf-8') as f:
    json.dump(rows, f, indent=4, ensure_ascii=False)

print(f"'{csv_file}' 파일을 '{json_file}'로 변환했습니다.")


'preprocessed_data(18)/user.csv' 파일을 'preprocessed_data(18)/users.json'로 변환했습니다.
